# Question: How to use the backend calibrations maintained by IBM 2 calculate the Cross Cross Resonance pulse parameters?

## 1.  Collection of Provided Values

step 1: collect the values we can get from the maintained calibrations 

In [2]:
import pandas as pd
import os
csv_path = os.getcwd() + "\\ibmq_casablanca_calibrations_2021_04_06.csv"
df = pd.read_csv(csv_path)
print(df.columns)
df

Index(['Qubit', 'T1 (us)', 'T2 (us)', 'Frequency (GHz)', 'Anharmonicity (GHz)',
       'Readout assignment error ', 'Prob meas0 prep1 ', 'Prob meas1 prep0 ',
       'Readout length (ns)', 'ID error ', '√x (sx) error ',
       'Single-qubit Pauli-X error ', 'CNOT error ', 'Gate time (ns)'],
      dtype='object')


,Qubit,T1 (us),T2 (us),Frequency (GHz),Anharmonicity (GHz),Readout assignment error,Prob meas0 prep1,Prob meas1 prep0,Readout length (ns),ID error,√x (sx) error,Single-qubit Pauli-X error,CNOT error,Gate time (ns)
0,Q0,98.23,57.33,4.822,-0.33990,0.0398,0.0560,0.0236,5351.111,0.000272,0.000272,0.000272,0_1:7.774e-3,0_1:497.778
1,Q1,125.07,125.85,4.760,-0.32644,0.0143,0.0162,0.0124,5351.111,0.000182,0.000182,0.000182,1_3:5.189e-3; 1_0:7.774e-3; 1_2:1.296e-2,1_3:355.556; 1_0:462.222; 1_2:398.222
2,Q2,107.36,131.63,4.907,-0.33862,0.0167,0.0186,0.0148,5351.111,0.002543,0.002543,0.002543,2_1:1.296e-2,2_1:433.778
3,Q3,93.69,156.47,4.879,-0.33864,0.0179,0.0282,0.0076,5351.111,0.000195,0.000195,0.000195,3_5:7.657e-3; 3_1:5.189e-3,3_5:433.778; 3_1:320
4,Q4,123.99,52.08,4.871,-0.33889,0.0242,0.0342,0.0142,5351.111,0.000316,0.000316,0.000316,4_5:7.640e-3,4_5:398.222
5,Q5,59.04,10.46,4.964,-0.32184,0.0189,0.0298,0.0080,5351.111,0.000341,0.000341,0.000341,5_6:1.522e-2; 5_4:7.640e-3; 5_3:7.657e-3,5_6:341.333; 5_4:362.667; 5_3:398.222
6,Q6,102.03,142.38,5.177,-0.33499,0.0184,0.0276,0.0092,5351.111,0.000222,0.000222,0.000222,6_5:1.522e-2,6_5:305.778


In [3]:
from qiskit import pulse
import numpy as np
import matplotlib.pyplot as plt
from qiskit.pulse import Play, Schedule, DriveChannel, ControlChannel, Waveform
from qiskit.pulse.library import drag, GaussianSquare
from qiskit import QuantumCircuit, Aer, IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')

backend_config = backend.configuration()
backend_defaults = backend.defaults()

print(backend_defaults)

C:\ProgramData\Anaconda3\envs\walter\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<PulseDefaults(<InstructionScheduleMap(1Q instructions:
  q0: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q1: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q2: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q3: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q4: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q5: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
  q6: {'u1', 'measure', 'u2', 'u3', 'rz', 'sx', 'x', 'id'}
Multi qubit instructions:
  (0, 1): {'cx'}
  (1, 0): {'cx'}
  (1, 2): {'cx'}
  (1, 3): {'cx'}
  (2, 1): {'cx'}
  (3, 1): {'cx'}
  (3, 5): {'cx'}
  (4, 5): {'cx'}
  (5, 3): {'cx'}
  (5, 4): {'cx'}
  (5, 6): {'cx'}
  (6, 5): {'cx'}
  (0, 1, 2, 3, 4, 5, 6): {'measure'}
)>Qubit Frequencies [GHz]
[4.822100492352904, 4.759824933715048, 4.9073428582263325, 4.878972058210034, 4.8709098053914754, 4.963893057649185, 5.177096795422384]
Measurement Frequencies [GHz]
[7.284054663, 7.385438514, 7.32318003, 7.230756417, 7.280310863, 7.14699

In [4]:
dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns") 

Sampling time: 0.2222222222222222 ns


In [5]:
gates_config = backend_config.gates 
print(gates_config )

[GateConfig(id, [], gate id q { U(0, 0, 0) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(rz, ['theta'], gate rz(theta) q { U(0, 0, theta) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(sx, [], gate sx q { U(pi/2, 3*pi/2, pi/2) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(x, [], gate x q { U(pi, 0, pi) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(cx, [], gate cx q0, q1 { CX q0, q1; }, [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]), GateConfig(reset, None, None)]


In [11]:
channels  = backend_config.channels  
print(channels.keys())

dict_keys(['acquire0', 'acquire1', 'acquire2', 'acquire3', 'acquire4', 'acquire5', 'acquire6', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'u0', 'u1', 'u10', 'u11', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7', 'u8', 'u9'])


In [7]:
system_hamiltonian = backend_config.hamiltonian 
type(system_hamiltonian)

dict

In [8]:
system_hamiltonian_params = backend_config.hamiltonian['vars']
print(system_hamiltonian_params)

{'delta0': -2135684895.7375534, 'delta1': -2051069517.9330974, 'delta2': -2127640392.4573247, 'delta3': -2127748155.8240955, 'delta4': -2129301739.398853, 'delta5': -2022190613.7524579, 'delta6': -2104827921.7467928, 'jq0q1': 8155201.12104726, 'jq1q2': 8960302.871474529, 'jq1q3': 8704187.306888234, 'jq3q5': 8755759.807187283, 'jq4q5': 9192358.915922279, 'jq5q6': 10792387.809142636, 'omegad0': 1429985021.1747696, 'omegad1': 1041603589.3202244, 'omegad2': 1410465124.990575, 'omegad3': 1545264670.423624, 'omegad4': 1488038137.8237498, 'omegad5': 1080975279.6225054, 'omegad6': 915531089.4933453, 'wq0': 30298150963.295216, 'wq1': 29906862088.26544, 'wq2': 30833744544.10037, 'wq3': 30655485550.285034, 'wq4': 30604828921.832703, 'wq5': 31189059926.23211, 'wq6': 32528658518.844444}


In [65]:
inst_sched_map_instructions = inst_sched_map.instructions
print(inst_sched_map_instructions)

['cx', 'id', 'measure', 'rz', 'sx', 'u1', 'u2', 'u3', 'x']


In [68]:
n_uchannels = backend_config.n_uchannels
print(n_uchannels)

12


In [66]:
u_channel_lo = backend_config.u_channel_lo
print(u_channel_lo)

[[UchannelLO(1, (1+0j))], [UchannelLO(0, (1+0j))], [UchannelLO(2, (1+0j))], [UchannelLO(3, (1+0j))], [UchannelLO(1, (1+0j))], [UchannelLO(1, (1+0j))], [UchannelLO(5, (1+0j))], [UchannelLO(5, (1+0j))], [UchannelLO(3, (1+0j))], [UchannelLO(4, (1+0j))], [UchannelLO(6, (1+0j))], [UchannelLO(5, (1+0j))]]


In [67]:
qubit_lo_range = backend_config.qubit_lo_range 
print(qubit_lo_range )

[[4322067824.00621, 5322067824.00621], [4259858136.331707, 5259858136.331707], [4407299973.647209, 5407299973.647209], [4378972802.917337, 5378972802.917337], [4370905406.405776, 5370905406.405776], [4463897856.016808, 5463897856.016808], [4677097674.634708, 5677097674.634708]]


In [70]:
meas_lo_range = backend_config.meas_lo_range 
print(meas_lo_range )

[[6784054663.0, 7784054663.0], [6885438514.0, 7885438514.0], [6823180030.0, 7823180030.0], [6730756417.0, 7730756417.0], [6780310863.0, 7780310863.0], [6646993373.0, 7646993373.0], [6900652534.0, 7900652534.0]]


In [69]:
meas_levels = backend_config.meas_levels
print(meas_levels)

[1, 2]


In [71]:
def cx_pulse_instructions(qc: int, qt: int) -> Schedule:
    """Retrieve the CNOT pulse schedule for the given
    qubit pair from the backend defaults.
    
    Args:
      qc: control qubit index
      qt: target qubit index
    """
    if [qc, qt] not in backend_config.coupling_map:
        print('Qubit pair has no direct cross resonance!')
    else:
        cx = inst_sched_map.get('cx', qubits=[qc, qt])
        return cx
    
def get_cr_chan_inst(qc: int, qt: int) -> (ControlChannel, Waveform):
    """Retrieve the cross resonance pulse channel and waveform
    for the qubit pair from the cx (CNOT) schedule.
    
    Args:
      qc: control qubit index
      qt: target qubit index
    """
    cx = cx_pulse_instructions(qc, qt)
    idx = 0
    # look for first Play instruction on a ControlChannel
    while (type(cx.instructions[idx][1].channels[0]) is not ControlChannel) or \
        (type(cx.instructions[idx][1]) is not Play):
        idx += 1
    return (cx.instructions[idx][1].channels[0], cx.instructions[idx][1].pulse)

In [97]:
print(get_cr_chan_inst(0, 1))
print(get_cr_chan_inst(0, 1)==get_cr_chan_inst(0, 1))

(ControlChannel(1), GaussianSquare(duration=880, amp=(0.09491427571513351+0.20055835852752296j), sigma=64, width=624))
True


In [98]:
print(cx_pulse_instructions(0, 1),"\n")
print(cx_pulse_instructions(1, 0))

Schedule((0, ShiftPhase(-3.141592653589793, DriveChannel(0))), (0, ShiftPhase(-1.5707963267948966, DriveChannel(1))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(0))), (0, ShiftPhase(-3.141592653589793, ControlChannel(1))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(4))), (0, ShiftPhase(-1.5707963267948966, ControlChannel(5))), (0, Play(Drag(duration=160, amp=(-0.0009081373423082974+0.06603496709836688j), sigma=40, beta=-0.34556785501122383), DriveChannel(0))), (0, Play(Drag(duration=160, amp=(0.09081596827880498+0.0014195310134645113j), sigma=40, beta=-0.5394615377493713), DriveChannel(1))), (160, Play(GaussianSquare(duration=880, amp=(0.023928903710885352+0.0005715711085036445j), sigma=64, width=624), DriveChannel(0))), (160, Play(GaussianSquare(duration=880, amp=(0.09491427571513351+0.20055835852752296j), sigma=64, width=624), ControlChannel(1))), (1040, Play(Drag(duration=160, amp=(0.1817420085362087+0j), sigma=40, beta=-0.6193790259524857), DriveChannel(1))), (1200,

In [99]:
get_cr_chan_inst(1, 2)

(ControlChannel(2),
 GaussianSquare(duration=736, amp=(0.7546130757045991+0.3016659850455465j), sigma=64, width=480))

In [100]:
get_cr_chan_inst(1, 3)

(ControlChannel(5),
 GaussianSquare(duration=560, amp=(-0.32494487484900486-0.16257580959444107j), sigma=64, width=304))

In [101]:
get_cr_chan_inst(3, 5)

(ControlChannel(8),
 GaussianSquare(duration=736, amp=(0.20425006279967148-0.09325119936682269j), sigma=64, width=480))

In [102]:
get_cr_chan_inst(4, 5)

(ControlChannel(9),
 GaussianSquare(duration=656, amp=(-0.23464071022093874+0.12436373379320753j), sigma=64, width=400))

In [103]:
get_cr_chan_inst(5, 6)

(ControlChannel(11),
 GaussianSquare(duration=528, amp=(0.13907902732829266+0.4175197257727899j), sigma=64, width=272))